# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.shape

(4870, 65)

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [17]:
df_b = data[data.race =='b']
df_w = data[data.race =='w']

(2435, 65)
(157, 65)


Question1 What test is appropriate for this problem? Does CLT apply?
Two sample proportion z test is appropriate for this problem. The samples meet the following three condifitons:
1. the samples are randomly picked
2. the samples are independent (sampling wihtout replacement or n<10% of population)
3. the samples are normally distributed (at lease 10 successes and 10 failuures in each sample)

Both two samples have sample size > 30. CLT will apply to both samples.

Question2 What are the null and alternate hypotheses?
Null hypothesis: There is no difference between the population proportion of call back for both black and white races 
Alternative hypothesis: THere is difference between the population proportion of call back for black and white races


In [23]:
# Use bootstrap method to test
# Mean of callback is actually the proportion of callback
df_w = data[data.race=='w'].call.values
df_b = data[data.race=='b'].call.values
print(np.mean(df_w))
print(np.mean(df_b))
diff_of_means = np.mean(df_w) - np.mean(df_b)
print(diff_of_means)

0.09650924
0.064476386
0.032032855


In [110]:
# Define diff_from_mean , permutation_sample and draw_perm_reps functions for bootstrap
def diff_from_means(data_1, data_2):
    """Get abs value of mean difference between two dataset."""
    return np.mean(data_1) - np.mean(data_2)

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""
    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))
    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)
    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""
    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

In [111]:
# Generate replicates
perm_replicates = draw_perm_reps(df_w, df_b, diff_from_means, size=10000)

# Compute pvalue (pvalue is 0, we need to reject the null hypothesis)
p_value = np.sum(perm_replicates >= diff_of_means) / len(perm_replicates)
print(p_value)

# Compute 95% confidence interval of population mean difference
conf_int_boot = np.percentile(perm_replicates, [2.5,97.5])
print(conf_int_boot)

0
[-0.01560576  0.01560576]


In [106]:
# Use frequentist statistical approaches
prop_b = np.mean(data[data.race=='b'].call.values)
n_b = len(data[data.race=='b'])
prop_w = np.mean(data[data.race=='w'].call.values)
n_w = len(data[data.race=='w'])

# Use sample proportion to estimate std for prop_w - prop_b distribution
sigma = np.sqrt( prop_w*(1-prop_w)/n_w + prop_b*(1-prop_b)/n_b )

# Compute Margin of Error
moe = 1.96 * sigma

# Compute 95% confidence interval for the difference proportion of white and black callback
conf_int_freq = prop_w - prop_b + np.array([-1, 1]) * moe
print(conf_int_freq)

[0.01677745 0.04728826]


In [115]:
# Test null hypothesis using frequentist statistical approaches
# If we assuem both races have the same proportion of call back, we need to use the proportion of callbacks in the dataset
p = np.mean(data.call.values)
print("total proportion: {}".format(p))

# Under the null hypothese, the difference proportion of these two races have mean of 0, and std of sigma_null
# We use p to estimate sigma_null
sigma_null = np.sqrt(p*(1-p)/n_w + p*(1-p)/n_b)
print("sigma for null hypothesis: {}".format(sigma_null))

z_score = ((prop_w - prop_b) - 0 )/sigma_null
print("z score for null hypothesis: {}".format(z_score))

total proportion: 0.0804928168654
sigma for null hypothesis: 0.00779689420073
z score for null hypothesis: 4.10841214853


Question4: Write a story describing the statistical significance in the context or the original problem.

We have the null hypothesis stating that there is no difference in proportion of call back between two races (white and black). Under the null hypothesis, I calculate the possibility of getting the values at least as extrem as the observed proportion difference (0.032032855). The possiblity (p-value) turnes out to be very small (<0.05). Based on p-value, we should reject the null hypothesis. There are strong evidents to support that the proportions of callback are significantly different between two races.

Question5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis does not consider other factors, it only looks at race and callbacks. Race may be a fact that contributes to callback proportion. But there are some other factors in the data we need to consider. For example, education and years of experience can contribute in the employers' callback decisions. We may want to exam the corelations between different features (besides race) and callback rate, and we can find the dominant factor that contribute most to callback rate. 